# Overview

This notebook shows you how to put Landsat data into an AWS deployment of the Open Data Cube.

## Credentials

We use the Boto3 library to work with AWS. Saving data to S3 requires credentials with permission to put data into an S3 bucket. Obviously, if you're contributing to this notebook, you shouldn't commit credentials to the repository. 

In [58]:
import boto3

In [59]:
s3 = boto3.resource('s3')
bucket_name = 'test-odc-bucket'
bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(obj.key)

LT05_L1TP_043027_19860805_20161004_01_T1.xml
LT05_L1TP_043027_19860805_20161004_01_T1_ANG.txt
LT05_L1TP_043027_19860805_20161004_01_T1_GCP.txt
LT05_L1TP_043027_19860805_20161004_01_T1_MTL.txt
LT05_L1TP_043027_19860805_20161004_01_T1_VER.jpg
LT05_L1TP_043027_19860805_20161004_01_T1_VER.txt
LT05_L1TP_043027_19860805_20161004_01_T1_bqa.tif
LT05_L1TP_043027_19860805_20161004_01_T1_cfmask.tif
LT05_L1TP_043027_19860805_20161004_01_T1_cfmask_conf.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_atmos_opacity.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band1.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band2.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band3.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band4.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band5.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band7.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_cloud_qa.tif
LT05_L1TP_043027_19860805_20161004_01_T1_toa_band1.tif
LT05_L1TP_043027_19860805_20161004_01_T1_toa_band2.tif

KeyboardInterrupt: 

## 2. Saving a Scene

In [3]:
import requests
import gzip
import tarfile
import io

In [4]:
def download(url):
    """"""
    r = requests.get(url, stream=True)
    r.raw.decode_content = True
    data = io.BytesIO(r.raw)
    return data

In [5]:
def expand_and_put(path):
    """"""
    with tarfile.open(path) as tar:
        for name in tar.getnames():
            print(name)
            data = tar.extractfile(name).read()
            bucket.put_object(Key=name, Body=data)

In [ ]:
url = 'data/foo.tar.gz'
data = expand_and_put(url)

## 3. Build a Dataset YAML

In [6]:
import re
import uuid
import dateutil
from datetime import timedelta
from xml.etree import ElementTree
ns = {
    'espa': 'http://espa.cr.usgs.gov/v2'
}

In [7]:
def get_s3_url(bucket_name, obj_key):
    return 's3://{bucket_name}/{obj_key}'.format(
        bucket_name=bucket_name, 
        obj_key=obj_key)

In [8]:
def get_defined_metadata(url):
    fields = {
        'id': str(uuid.uuid5(uuid.NAMESPACE_URL, url)),
        'processing_level': 'ARD',
        'product_type': 'LS_USGS_ARD',
    }
    return fields

In [9]:
def parse_product_id(key):
    fields = re.match(
    (
        r"(?P<code>LC08|LE07|LT05|LT04)_"
        r"(?P<processing_level>L1TP|L1GT|L1GS)_"
        r"(?P<path>[0-9]{3})(?P<row>[0-9]{3})_"
        r"(?P<acquisition_year>[0-9]{4})(?P<acquisition_month>[0-9]{2})(?P<acquisition_day>[0-9]{2})_"
        r"(?P<processing_year>[0-9]{4})(?P<processing_month>[0-9]{2})(?P<processing_day>[0-9]{2})_"
        r"(?P<collection_number>[0-9]{2})_"
        r"(?P<tier>\w+)"
    ), key).groupdict()
    
    fields['processing_date'] = '{processing_year}-{processing_month}-{processing_day}'.format(**fields)
    
    return fields

In [10]:
def parse_mtl(xml_doc):
    lpgs_metadata_file = xml_doc.find('.//espa:lpgs_metadata_file', ns).text
    #TODO: Actually read this file
    return {'groundstation': 'XXX'}

In [11]:
import rasterio
from rasterio.errors import RasterioIOError
import rasterio.features
import shapely.affinity
import shapely.geometry
import shapely.ops

def safe_valid_region(images, mask_value=None):
    try:
        return valid_region(images, mask_value)
    except (OSError, RasterioIOError):
        return None


def valid_region(images, mask_value=None):
    mask = None

    for fname in images:
        # ensure formats match
        with rasterio.open(str(fname), 'r') as ds:
            transform = ds.transform
            img = ds.read(1)

            if mask_value is not None:
                new_mask = img & mask_value == mask_value
            else:
                new_mask = img != ds.nodata
            if mask is None:
                mask = new_mask
            else:
                mask |= new_mask

    shapes = rasterio.features.shapes(mask.astype('uint8'), mask=mask)
    shape = shapely.ops.unary_union([shapely.geometry.shape(shape) for shape, val in shapes if val == 1])

    geom = shape.convex_hull
    geom = geom.buffer(1, join_style=3, cap_style=3)
    geom = geom.simplify(1)
    geom = geom.intersection(shapely.geometry.box(0, 0, mask.shape[1], mask.shape[0]))

    # transform from pixel space into CRS space
    geom = shapely.affinity.affine_transform(geom, (transform.a, transform.b, transform.d,
                                                    transform.e, transform.xoff, transform.yoff))

    output = shapely.geometry.mapping(geom)
    output['coordinates'] = _to_lists(output['coordinates'])
    return output

In [12]:
def parse_xml_metadata(xmlDoc):
    fields = {}
    fields['product_id'] = xmlDoc.find('.//espa:product_id', ns).text
    fields.update(parse_product_id(fields['product_id']))
    
    fields['satellite'] = xmlDoc.find('.//espa:satellite', ns).text
    fields['instrument'] = xmlDoc.find('.//espa:instrument', ns).text
    
    acquisition_date = xmlDoc.find('.//espa:acquisition_date', ns).text.replace("-", "")
    scene_center_time = xmlDoc.find('.//espa:scene_center_time', ns).text[:8]
    
    center_dt = dateutil.parser.parse(acquisition_date + "T" + scene_center_time)
    aos = dateutil.parser.parse(acquisition_date + "T" + scene_center_time) - timedelta(seconds=(24 / 2))
    los = aos + timedelta(seconds=24)
    fields['start_time'] = str(aos)
    fields['end_time'] = str(los)
    fields['center_dt'] = str(center_dt)
    
    fields['creation_dt'] = str(dateutil.parser.parse(xmlDoc.find('.//espa:level1_production_date', ns).text))
    
    pr = xmlDoc.find('.//espa:wrs', ns)
    fields['path'] = pr.attrib['path']
    fields['row'] = pr.attrib['row']
    
    return fields

In [13]:
def lookup_band_name(band):
    band_name = band.attrib['name']
    # TODO: Simon wants to rename the bands
#     band_lookup = {
#         'bqa': '',
#         'toa_band1': '',
#         'toa_band2': '',
#         'toa_band3': '',
#         'toa_band4': '',
#         'toa_band5': '',
#         'toa_band6': '',
#         'toa_band7': '',
#         'toa_band6_qa': '',
#         'toa_qa': '',
#         'sr_band1': '',
#         'sr_band2': '',
#         'sr_band3': '',
#         'sr_band4': '',
#         'sr_band5': '',
#         'sr_band7': '',

#         'sr_atmos_opacity': '',
#         'sr_cloud_qa': '',
#         'cfmask': '',
#         'cfmask_conf': '',
#     }
#     return band_lookup.get(band_name, band_name)
    return band_name


def get_bands(xml_doc):
    bands = xml_doc.findall('.//espa:band', ns)
    band_dict = {lookup_band_name(band): get_s3_url(bucket_name, band.find('.//espa:file_name', ns).text) 
                 for band in bands}
    return band_dict

In [ ]:
xml_doc = ElementTree.parse('/home/andrew/Data/usgs/LT050430271984116-SC20170101123907/LT05_L1TP_043027_19840425_20161004_01_T1.xml')
west = float(xml_doc.find('.//espa:bounding_coordinates/espa:west', ns).text)
east = float(xml_doc.find('.//espa:bounding_coordinates/espa:east', ns).text)
north = float(xml_doc.find('.//espa:bounding_coordinates/espa:north', ns).text)
south = float(xml_doc.find('.//espa:bounding_coordinates/espa:south', ns).text)
north, south, east, west

In [14]:
def get_projection(path):
    with rasterio.open(str(path)) as img:
        left, bottom, right, top = img.bounds
        return {
            'spatial_reference': str(str(getattr(img, 'crs_wkt', None) or img.crs.wkt)),
            'geo_ref_points': {
                'ul': {'x': left, 'y': top},
                'ur': {'x': right, 'y': top},
                'll': {'x': left, 'y': bottom},
                'lr': {'x': right, 'y': bottom},
            }
        }

In [15]:
from osgeo import osr
def get_coords(spatial_reference, geo_ref_points):
    spatial_ref = osr.SpatialReference(spatial_reference)
    t = osr.CoordinateTransformation(spatial_ref, spatial_ref.CloneGeogCS())

    def transform(p):
        lon, lat, z = t.TransformPoint(p['x'], p['y'])
        return {'lon': lon, 'lat': lat}
    return {key: transform(p) for key, p in geo_ref_points.items()}

In [16]:
def make_metadata_doc(fields):
    doc = {
        'id': fields['id'],
        'processing_level': fields["processing_level"],
        'product_type': fields["product_type"],
        'creation_dt': fields["creation_dt"],
        'platform': {'code': fields['satellite']},
        'instrument': {'name': fields["instrument"]},
        'acquisition': {
            'groundstation': {
                'code': fields['groundstation'],    
            },
            'aos': fields["start_time"],
            'los': fields["end_time"],
        },
        'extent': {
            'from_dt': fields["start_time"],
            'to_dt': fields["end_time"],
            'center_dt': fields["center_dt"],
            'coord': fields['coord'],
        },
        'format': {'name': 'GeoTiff'},
        'grid_spatial': {
            'projection': fields['projection'] 
        },
        'image': {
            'satellite_ref_point_start': {'x': int(fields["path"]), 'y': int(fields["row"])},
            'satellite_ref_point_end': {'x': int(fields["path"]), 'y': int(fields["row"])},
            'bands': {key: {'path': value} for (key, value) in fields['bands'].items()},
        },

        'lineage': {'source_datasets': {}}
    }
    return doc

In [60]:
def get_metadata_docs(bucket):
    for obj in bucket.objects.all():
        if obj.key[-3:].lower() == 'xml':
            fields = get_defined_metadata(get_s3_url(bucket.name, obj.key))
            if index.datasets.has(fields['id']):
                continue
                
            response = obj.get()
            body = response['Body']
            xml_doc = ElementTree.parse(body)

            fields.update(parse_xml_metadata(xml_doc))

            bands = get_bands(xml_doc)
            fields['bands'] = bands
            sample_band = next(iter(bands.values()))
            projection = get_projection(sample_band)
            fields['projection'] = projection
            fields['coord'] = get_coords(**projection)
            fields.update(parse_mtl(xml_doc))

            metadata_doc = make_metadata_doc(fields)
            yield obj.key, metadata_doc

## 4. Build an Index

In [ ]:
!datacube system init --no-init-users

In [ ]:
!datacube product add ~/Projects/datacube-core/docs/config_samples/dataset_types/usgs_ard.yaml

In [ ]:
!datacube product list

In [20]:
%matplotlib inline
import datacube
dc = datacube.Datacube()

In [21]:
dc.list_products()

,name,description,platform,instrument,lat,product_type,time,format,lon,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
1,ls5_usgs_ard,Landsat 5 USGS ARD 30 metre tile,LANDSAT_5,TM,None,LS_USGS_ARD,None,GeoTiff,None,"PROJCS[""Albers"", GEOGCS[""NAD83"", DATUM[""North_...","[-30, 30]",None,"(y, x)"


In [22]:
from datacube.scripts.dataset import create_dataset, load_rules_from_types
index = dc.index

product = index.products.get_by_name('ls5_usgs_ard')
product

DatasetType(name='ls5_usgs_ard', id_=1)

For each xml file in the S3 bucket, create a dataset object and add it to the index.

In [61]:
for metadata_path, metadata_doc in get_metadata_docs(bucket):
    uri = get_s3_url(bucket.name, metadata_path)
    print(metadata_path)
    d = datacube.model.Dataset(product, metadata_doc, local_uri=uri, sources={})
    index.datasets.add(d)

LT05_L1TP_043027_19900325_20161001_01_T1.xml
LT05_L1TP_043027_19900410_20161001_01_T1.xml
LT05_L1TP_043027_19900426_20161002_01_T1.xml
LT05_L1TP_043027_19900629_20161001_01_T1.xml
LT05_L1TP_043027_19900715_20161002_01_T1.xml
LT05_L1TP_043027_19900731_20161002_01_T1.xml
LT05_L1TP_043027_19900816_20160930_01_T1.xml
LT05_L1TP_043027_19900901_20161002_01_T1.xml
LT05_L1TP_043027_19900917_20160930_01_T1.xml
LT05_L1TP_043027_19901019_20161002_01_T1.xml
LT05_L1TP_043027_19930112_20160928_01_T1.xml
LT05_L1TP_043027_19930301_20160928_01_T1.xml
LT05_L1TP_043027_19930418_20160928_01_T1.xml
LT05_L1TP_043027_19930520_20160928_01_T1.xml
LT05_L1TP_043027_19930621_20160928_01_T1.xml
LT05_L1TP_043027_19930707_20160928_01_T1.xml
LT05_L1TP_043027_19930808_20160927_01_T1.xml
LT05_L1TP_043027_19930824_20160928_01_T1.xml
LT05_L1TP_043027_19930909_20160927_01_T1.xml
LT05_L1TP_043027_19930925_20160927_01_T1.xml
LT05_L1TP_043027_19931011_20160927_01_T1.xml
LT05_L1TP_043027_19931027_20160927_01_T1.xml
LT05_L1TP_

## 5. Loading some data

In [25]:
data = dc.load('ls5_usgs_ard', time=('1991-03', '1991-04'), dask_chunks={'time':1, 'x': 5000, 'y': 1})

In [26]:
data.sizes

Frozen(SortedKeysDict({'time': 6, 'y': 5001, 'x': 5001}))

In [27]:
data.time.values

array(['1991-03-17T18:16:41.000000000', '1991-03-17T18:17:05.000000000',
       '1991-03-19T18:04:47.000000000', '1991-03-26T18:10:39.000000000',
       '1991-03-26T18:11:02.000000000', '1991-03-26T18:11:26.000000000'], dtype='datetime64[ns]')

In [28]:
import dask
import dask.multiprocessing

In [ ]:
%%time
spaghetti1 = data.sr_band1[:,2500:2650, 2500:2650]
with dask.set_options(get=dask.async.get_sync):
    spaghetti1.load()

In [ ]:
%%time
spaghetti2 = data.sr_band1[:,2500:2650, 2500:2650]
with dask.set_options(get=dask.multiprocessing.get):
    spaghetti2.load()

In [ ]:
%%time
data.sr_band1[2].plot.imshow()

In [ ]:
data.sr_band1[2].data.visualize()

In [48]:
bands = dc.list_measurements().loc['ls5_usgs_ard']
bands[bands['aliases'].apply(lambda b: 'red' in b)]

,aliases,dtype,flags_definition,name,nodata,spectral_definition,units
measurement,,,,,,,
sr_band3,"[band_3, red]",int16,NaN,sr_band3,-9999,"{'response': [0.0018, 0.002, 0.0079, 0.0205, 0...",1


In [49]:
%%time 
data_1 = dc.load('ls5_usgs_ard', measurements=['sr_band3'], time=('1991-03-18', '1991-03-20'))

CPU times: user 416 ms, sys: 464 ms, total: 880 ms
Wall time: 17.6 s


In [53]:
data_1.sr_band3.plot.imshow()

ValueError: DataArray must be 2d

In [32]:
%%time 
with rasterio.Env(CPL_DEBUG=True, 
                  GDAL_DISABLE_READDIR_ON_OPEN=True, 
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='tif'):
    data_2 = dc.load('ls5_usgs_ard', measurements=['sr_band3'], time=('1991-03-18', '1991-03-20'))

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/home/andrew/miniconda3/envs/datacube/lib/python3.6/logging/__init__.py", line 1811, in getLogger
    def getLogger(name=None):
KeyboardInterrupt
Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/home/andrew/miniconda3/envs/datacube/lib/python3.6/logging/__init__.py", line 1811, in getLogger
    def getLogger(name=None):
KeyboardInterrupt
Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/home/andrew/miniconda3/envs/datacube/lib/python3.6/logging/__init__.py", line 1811, in getLogger
    def getLogger(name=None):
KeyboardInterrupt


CPU times: user 7.95 s, sys: 6.99 s, total: 14.9 s
Wall time: 3min 47s


In [30]:
data.sizes

Frozen(SortedKeysDict({'time': 3, 'y': 5001, 'x': 5001}))

In [31]:
import rasterio